<a href="https://colab.research.google.com/github/Natdata/Reading_calculator/blob/main/Kalkulator_przeczyta%C5%84.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [38]:
start = '2024-03-05'
end = '2024-03-14'
value = ['349996']
numbers_of_pages = 43


In [39]:
from datetime import datetime, timedelta
import requests
import json
import pandas as pd
import time

event_name = 'leaflet_quit_full'
name = 'leaflet_id'
operator = 'equals'

start = start.replace('-', '')
start = int(start)

end = end.replace('-', '')
end = int(end)

headers = {
    'X-CleverTap-Account-Id': 'R95-8KW-ZR5Z',
    'X-CleverTap-Passcode': '7dd63a92-b9c9-40c3-a004-330f9739e431',
    'Content-Type': 'application/json',
}

data = {
    "event_name": event_name,
    "event_properties": [{"name": name, "operator": operator, "value": value}],
    "from": start,
    "to": end
}

results = []  # Lista do przechowywania wyników
json_data = json.dumps(data)
response = requests.post('https://eu1.api.clevertap.com/1/counts/events.json', headers=headers, data=json_data)
response_json = response.json()

req_id = response_json['req_id']

    # Odczekanie 5 sekund przed wykonaniem kolejnego żądania GET
time.sleep(3)

response1 = requests.get(
        f'https://eu1.api.clevertap.com/1/counts/events.json?req_id={req_id}', headers=headers)
response1_json = response1.json()

    # Dodawanie wyników do listy jako słownik
result = {

        "Count": response1_json.get("count", 0)
    }
results.append(result)

# Tworzenie DataFrame z wyników
Leaflet_quit_full_count = pd.DataFrame(results)

print(Leaflet_quit_full_count)
print()





   Count
0   3876



In [32]:
event_name2 = 'leaflet_quit_part'


results = []  # Lista do przechowywania wyników
page_range = 0  # Początkowy zakres stron

while True:
    print(f"Page range is now: '{page_range}'")  # Wypisywanie aktualnego zakresu stron
    data = {
        "event_name": event_name2,
        "event_properties": [{"name": name, "operator": operator, "value": value },{"name": "seen_pages_count", "operator": operator, "value": page_range}],
        "from": start,
        "to": end
    }
    json_data = json.dumps(data)
    print(f"Sending data: {json_data}")  # Debug: Wydrukuj wysyłane dane
    response = requests.post('https://eu1.api.clevertap.com/1/counts/events.json', headers=headers, data=json_data)
    response_json = response.json()

    # Sprawdzanie, czy odpowiedź zawiera poprawne dane
    if 'req_id' in response_json:
        req_id = response_json['req_id']
        time.sleep(5)  # Odczekaj odpowiedni czas na przetworzenie żądania
        response1 = requests.get(f'https://eu1.api.clevertap.com/1/counts/events.json?req_id={req_id}', headers=headers)
        response1_json = response1.json()

        count = response1_json.get("count", 0)

        if count > 0:  # Jeśli są wyniki, kontynuuj
            result = {
                "Page range": page_range,
                "Count": count
            }
            results.append(result)
            print(f"Result for page range {page_range}: {count}")
            page_range += 1  # Przejdź do następnej strony
        else:
            break  # Przerwij pętlę, jeśli nie ma więcej wyników

df = pd.DataFrame(results)
print("Final results:")
print(df)

Page range is now: '0'
Sending data: {"event_name": "leaflet_quit_part", "event_properties": [{"name": "leaflet_id", "operator": "equals", "value": ["349996"]}, {"name": "seen_pages_count", "operator": "equals", "value": 0}], "from": 20240305, "to": 20240314}
Result for page range 0: 9
Page range is now: '1'
Sending data: {"event_name": "leaflet_quit_part", "event_properties": [{"name": "leaflet_id", "operator": "equals", "value": ["349996"]}, {"name": "seen_pages_count", "operator": "equals", "value": 1}], "from": 20240305, "to": 20240314}
Result for page range 1: 3740
Page range is now: '2'
Sending data: {"event_name": "leaflet_quit_part", "event_properties": [{"name": "leaflet_id", "operator": "equals", "value": ["349996"]}, {"name": "seen_pages_count", "operator": "equals", "value": 2}], "from": 20240305, "to": 20240314}
Result for page range 2: 279
Page range is now: '3'
Sending data: {"event_name": "leaflet_quit_part", "event_properties": [{"name": "leaflet_id", "operator": "equa

In [33]:
total_count_from_df = df['Count'].sum()

# Pobranie wartości 'Count' z Leaflet_quit_full_count
full_count = Leaflet_quit_full_count['Count'].iloc[0]

# Obliczenie zmiennej przeczytania
read_ratio = full_count / (total_count_from_df + full_count)
print(f"Zmienna przeczytania: {read_ratio * 100:.2f}%")
print(total_count_from_df)

Zmienna przeczytania: 31.84%
8298


In [34]:
new_df = df.copy()

# Modyfikacja 'Count' w nowym DataFrame
for index, row in new_df.iterrows():
    if row['Page range'] == numbers_of_pages:
        new_df.at[index, 'Count'] = full_count
filtered_new_df = new_df[(new_df['Page range'] >= 1) & (new_df['Page range'] <= numbers_of_pages)]
# Wynik końcowy - nowy DataFrame
print(filtered_new_df)

    Page range  Count
1            1   3740
2            2    279
3            3    246
4            4    149
5            5    122
6            6    107
7            7    105
8            8     76
9            9     70
10          10     67
11          11     49
12          12     58
13          13     50
14          14     56
15          15     52
16          16     37
17          17     66
18          18     30
19          19     27
20          20     41
21          21     42
22          22     51
23          23     41
24          24     60
25          25     41
26          26     46
27          27     53
28          28     47
29          29     41
30          30     39
31          31     29
32          32     34
33          33     38
34          34     36
35          35     48
36          36     38
37          37     63
38          38     95
39          39     94
40          40    334
41          41    897
42          42    680
43          43   3876


In [35]:
filtered_new_df['Cumulative Count'] = filtered_new_df['Count'][::-1].cumsum()[::-1]

# Tworzenie nowej tabeli z wykorzystaniem obliczonej kumulatywnej sumy
new_table = pd.DataFrame({
    'Page range': filtered_new_df['Page range'],
    'Count': filtered_new_df['Cumulative Count']
})

# Wynik końcowy - wyświetlenie nowej tabeli
print(new_table)

    Page range  Count
1            1  12150
2            2   8410
3            3   8131
4            4   7885
5            5   7736
6            6   7614
7            7   7507
8            8   7402
9            9   7326
10          10   7256
11          11   7189
12          12   7140
13          13   7082
14          14   7032
15          15   6976
16          16   6924
17          17   6887
18          18   6821
19          19   6791
20          20   6764
21          21   6723
22          22   6681
23          23   6630
24          24   6589
25          25   6529
26          26   6488
27          27   6442
28          28   6389
29          29   6342
30          30   6301
31          31   6262
32          32   6233
33          33   6199
34          34   6161
35          35   6125
36          36   6077
37          37   6039
38          38   5976
39          39   5881
40          40   5787
41          41   5453
42          42   4556
43          43   3876


<ipython-input-35-5499f1c6256e>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_new_df['Cumulative Count'] = filtered_new_df['Count'][::-1].cumsum()[::-1]


In [36]:
first_count = new_table['Count'].iloc[0]  # Pierwsza wartość Count
new_table['Percentage'] = new_table['Count'] / first_count * 100

# Formatowanie kolumny Percentage do dwóch miejsc po przecinku
new_table['Percentage'] = new_table['Percentage'].map('{:.2f}%'.format)

# Wynik końcowy - wyświetlenie nowej tabeli z procentami
print(new_table[['Page range', 'Percentage']])

    Page range Percentage
1            1    100.00%
2            2     69.22%
3            3     66.92%
4            4     64.90%
5            5     63.67%
6            6     62.67%
7            7     61.79%
8            8     60.92%
9            9     60.30%
10          10     59.72%
11          11     59.17%
12          12     58.77%
13          13     58.29%
14          14     57.88%
15          15     57.42%
16          16     56.99%
17          17     56.68%
18          18     56.14%
19          19     55.89%
20          20     55.67%
21          21     55.33%
22          22     54.99%
23          23     54.57%
24          24     54.23%
25          25     53.74%
26          26     53.40%
27          27     53.02%
28          28     52.58%
29          29     52.20%
30          30     51.86%
31          31     51.54%
32          32     51.30%
33          33     51.02%
34          34     50.71%
35          35     50.41%
36          36     50.02%
37          37     49.70%
38          

In [37]:
print(new_table[['Page range', 'Percentage']].to_string(index=False))

 Page range Percentage
          1    100.00%
          2     69.22%
          3     66.92%
          4     64.90%
          5     63.67%
          6     62.67%
          7     61.79%
          8     60.92%
          9     60.30%
         10     59.72%
         11     59.17%
         12     58.77%
         13     58.29%
         14     57.88%
         15     57.42%
         16     56.99%
         17     56.68%
         18     56.14%
         19     55.89%
         20     55.67%
         21     55.33%
         22     54.99%
         23     54.57%
         24     54.23%
         25     53.74%
         26     53.40%
         27     53.02%
         28     52.58%
         29     52.20%
         30     51.86%
         31     51.54%
         32     51.30%
         33     51.02%
         34     50.71%
         35     50.41%
         36     50.02%
         37     49.70%
         38     49.19%
         39     48.40%
         40     47.63%
         41     44.88%
         42     37.50%
         43